![arangodb](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/ArangoDB_logo.png?raw=1)

# AQL Join Tutorial

<a href="https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlJoinTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the previous parts of this AQL tutorial series, we have already covered a number of topics:
* [Part 1: CRUD](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb) 
* [Part 2: Limit, Sort, Filter](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlPart2Tutorial.ipynb)

In this part we cover how we can combine (aka join) information from multiple documents.


Recall, that in addition to the python interface, you can interact with ArangoDB using its web interface to manage collections and execute the queries.
This notebook creates a temporary database, so you can perform all queries in this notebook or use the URL provided to you below to access the web interface.

# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [0]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [0]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AqlJoinTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login["dbName"]] 

Requesting new temp credentials.
Temp database ready to use.


In [4]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTaoje52w1wqtlmwlw55aq9c
Password: TUTlt898z0uqkn0gibbyw41uyl
Database: TUT4b4475lmvq4ezzdpvpeumt


Feel free to use to above URL to checkout the UI!

##  Import Data

We will work with the same dataset as in the [first part of this tutorial](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb). So let us quickly import that into our temporary database. 

In [6]:
# Create the Characters Collection 
db.createCollection(name="Characters")

ArangoDB collection name: Characters, id: 14021944, type: document, status: loaded

In [7]:
insert_query = """
LET data = [
    { "name": "Ned", "surname": "Stark", "alive": true, "age": 41, "traits": ["A","H","C","N","P"] },
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

As before, let us check the `Characters` collection:

In [8]:
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ned

Robert

Jaime

Catelyn

Cersei

Daenerys

Jorah

Petyr

Viserys

Jon

Sansa

Arya

Robb

Theon

Bran

Joffrey

Sandor

Tyrion

Khal

Tywin

Davos

Samwell

Stannis

Melisandre

Margaery

Jeor

Bronn

Varys

Shae

Talisa

Gendry

Ygritte

Tormund

Gilly

Brienne

Ramsay

Ellaria

Daario

Missandei

Tommen

Jaqen

Roose

The High Sparrow



# Join

The character data we imported has an attribute traits for each character, which is an array of strings. It does not store character features directly however:

In [9]:
find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN {"Name": c.name, "Traits": c.traits}
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'Name': 'Ned', 'Traits': ['A', 'H', 'C', 'N', 'P']}



Traits are rather a list of letters without an apparent meaning. The idea here is that traits is supposed to store documents keys of another collection, which we can use to resolve the letters to labels such as “strong”. The benefit of using another collection for the actual traits is, that we can easily query for all existing traits later on and store labels in multiple languages for instance in a central place. If we would embed traits directly…

In [10]:
JSON({
    "Name": "Ned",
    "Traits": [
        {
            "de": "stark",
            "en": "strong"
        },
        {
            "de": "einflussreich",
            "en": "powerful"
        },
        {
            "de": "loyal",
            "en": "loyal"
        },
        {
            "de": "rational",
            "en": "rational"
        },
        {
            "de": "mutig",
            "en": "brave"
        }
    ]
})

<IPython.core.display.JSON object>

… it becomes really hard to maintain traits. If you were to rename or translate one of them, you would need to find all other character documents with the same trait and perform the changes there too. If we only refer to a trait in another collection, it is as easy as updating a single document.

![join](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/join.png?raw=1)

# Traits Collection

In [11]:
# Create the Characters Collection 
db.createCollection(name="Traits")

ArangoDB collection name: Traits, id: 14021950, type: document, status: loaded

In [12]:
insert_query = """
LET data = [
    { "_key": "A", "en": "strong", "de": "stark" },
    { "_key": "B", "en": "polite", "de": "freundlich" },
    { "_key": "C", "en": "loyal", "de": "loyal" },
    { "_key": "D", "en": "beautiful", "de": "schön" },
    { "_key": "E", "en": "sneaky", "de": "hinterlistig" },
    { "_key": "F", "en": "experienced", "de": "erfahren" },
    { "_key": "G", "en": "corrupt", "de": "korrupt" },
    { "_key": "H", "en": "powerful", "de": "einflussreich" },
    { "_key": "I", "en": "naive", "de": "naiv" },
    { "_key": "J", "en": "unmarried", "de": "unverheiratet" },
    { "_key": "K", "en": "skillful", "de": "geschickt" },
    { "_key": "L", "en": "young", "de": "jung" },
    { "_key": "M", "en": "smart", "de": "klug" },
    { "_key": "N", "en": "rational", "de": "rational" },
    { "_key": "O", "en": "ruthless", "de": "skrupellos" },
    { "_key": "P", "en": "brave", "de": "mutig" },
    { "_key": "Q", "en": "mighty", "de": "mächtig" },
    { "_key": "R", "en": "weak", "de": "schwach" }
]
  

FOR d IN data
    INSERT d INTO Traits
"""

db.AQLQuery(insert_query)

Let us quickly check the contents of the Traits collection:

In [13]:
all_traits = """
FOR t IN Traits
    RETURN t
"""

query_result = db.AQLQuery(all_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': 'A', '_id': 'Traits/A', '_rev': '_apll4-q---', 'en': 'strong', 'de': 'stark'}

{'_key': 'B', '_id': 'Traits/B', '_rev': '_apll4-q--_', 'en': 'polite', 'de': 'freundlich'}

{'_key': 'C', '_id': 'Traits/C', '_rev': '_apll4-q--A', 'en': 'loyal', 'de': 'loyal'}

{'_key': 'D', '_id': 'Traits/D', '_rev': '_apll4-q--C', 'en': 'beautiful', 'de': 'schön'}

{'_key': 'E', '_id': 'Traits/E', '_rev': '_apll4-q--D', 'en': 'sneaky', 'de': 'hinterlistig'}

{'_key': 'F', '_id': 'Traits/F', '_rev': '_apll4-q--E', 'en': 'experienced', 'de': 'erfahren'}

{'_key': 'G', '_id': 'Traits/G', '_rev': '_apll4-q--F', 'en': 'corrupt', 'de': 'korrupt'}

{'_key': 'H', '_id': 'Traits/H', '_rev': '_apll4-q--G', 'en': 'powerful', 'de': 'einflussreich'}

{'_key': 'I', '_id': 'Traits/I', '_rev': '_apll4-q--H', 'en': 'naive', 'de': 'naiv'}

{'_key': 'J', '_id': 'Traits/J', '_rev': '_apll4-q--I', 'en': 'unmarried', 'de': 'unverheiratet'}

{'_key': 'K', '_id': 'Traits/K', '_rev': '_apll4-q--J', 'en': 'skillful', 'd

# Joining Traits

Let’s start simple by returning only the traits attribute of each character:

In [14]:
all_characters_traits = """
FOR c IN Characters
    RETURN c.traits
"""

query_result = db.AQLQuery(all_characters_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

['A', 'H', 'C', 'N', 'P']

['A', 'H', 'C']

['A', 'F', 'B']

['D', 'H', 'C']

['H', 'E', 'F']

['D', 'H', 'C']

['A', 'B', 'C', 'F']

['E', 'G', 'F']

['O', 'L', 'N']

['A', 'B', 'C', 'F']

['D', 'I', 'J']

['C', 'K', 'L']

['A', 'B', 'C', 'K']

['E', 'R', 'K']

['L', 'J']

['I', 'L', 'O']

['A', 'P', 'K', 'F']

['F', 'K', 'M', 'N']

['A', 'C', 'O', 'P']

['O', 'M', 'H', 'F']

['C', 'K', 'P', 'F']

['C', 'L', 'I']

['H', 'O', 'P', 'M']

['G', 'E', 'H']

['M', 'D', 'B']

['C', 'H', 'M', 'P']

['K', 'E', 'C']

['M', 'F', 'N', 'E']

['M', 'D', 'G']

['D', 'C', 'B']

['K', 'C', 'A']

['A', 'P', 'K']

['C', 'P', 'A', 'I']

['L', 'J']

['P', 'C', 'A', 'K']

['E', 'O', 'G', 'A']

['P', 'O', 'A', 'E']

['K', 'P', 'A']

['D', 'L', 'C', 'M']

['I', 'L', 'B']

['H', 'F', 'K']

['H', 'E', 'F', 'A']

['H', 'M', 'F', 'O']



We can use the traits array together with the [DOCUMENT()](https://www.arangodb.com/docs/stable/aql/functions-miscellaneous.html#document) function to use the elements as document keys and look them up in the Traits collection.
The DOCUMENT() function can be used to look up a single or multiple documents via document identifiers. In our example, we pass the collection name from which we want to fetch documents as the first argument ("Traits") and an array of document keys (_key attribute) as the second argument. In the [RETURN](https://www.arangodb.com/docs/stable/aql/operations-return.html) statement we get an array of the full trait documents for each character.

In [15]:
all_characters_traits = """
FOR c IN Characters
    LIMIT 5
    RETURN DOCUMENT("Traits", c.traits)
"""

query_result = db.AQLQuery(all_characters_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

[{'_key': 'A', '_id': 'Traits/A', '_rev': '_apll4-q---', 'en': 'strong', 'de': 'stark'}, {'_key': 'H', '_id': 'Traits/H', '_rev': '_apll4-q--G', 'en': 'powerful', 'de': 'einflussreich'}, {'_key': 'C', '_id': 'Traits/C', '_rev': '_apll4-q--A', 'en': 'loyal', 'de': 'loyal'}, {'_key': 'N', '_id': 'Traits/N', '_rev': '_apll4-q--M', 'en': 'rational', 'de': 'rational'}, {'_key': 'P', '_id': 'Traits/P', '_rev': '_apll4-u---', 'en': 'brave', 'de': 'mutig'}]

[{'_key': 'A', '_id': 'Traits/A', '_rev': '_apll4-q---', 'en': 'strong', 'de': 'stark'}, {'_key': 'H', '_id': 'Traits/H', '_rev': '_apll4-q--G', 'en': 'powerful', 'de': 'einflussreich'}, {'_key': 'C', '_id': 'Traits/C', '_rev': '_apll4-q--A', 'en': 'loyal', 'de': 'loyal'}]

[{'_key': 'A', '_id': 'Traits/A', '_rev': '_apll4-q---', 'en': 'strong', 'de': 'stark'}, {'_key': 'F', '_id': 'Traits/F', '_rev': '_apll4-q--E', 'en': 'experienced', 'de': 'erfahren'}, {'_key': 'B', '_id': 'Traits/B', '_rev': '_apll4-q--_', 'en': 'polite', 'de': 'freund

This is a bit too much information, so let’s only return English labels using the array expansion notation:

In [16]:
all_characters_traits = """
FOR c IN Characters
    LIMIT 5
    RETURN DOCUMENT("Traits", c.traits)[*].en
"""

query_result = db.AQLQuery(all_characters_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

['strong', 'powerful', 'loyal', 'rational', 'brave']

['strong', 'powerful', 'loyal']

['strong', 'experienced', 'polite']

['beautiful', 'powerful', 'loyal']

['powerful', 'sneaky', 'experienced']



# Combining Characters and Traits

Great, we resolved the letters to meaningful traits! But we also need to know to which character they belong. Thus, we need to merge both the character document and the data from the trait documents:



In [17]:
all_characters_traits = """
FOR c IN Characters
    LIMIT 5
    RETURN MERGE(c, { traits: DOCUMENT("Traits", c.traits)[*].en } )
"""

query_result = db.AQLQuery(all_characters_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()    

{'_id': 'Characters/1026018918', '_key': '1026018918', '_rev': '_aplllqi---', 'age': 41, 'alive': True, 'name': 'Ned', 'surname': 'Stark', 'traits': ['strong', 'powerful', 'loyal', 'rational', 'brave']}

{'_id': 'Characters/1026018919', '_key': '1026018919', '_rev': '_aplllqi--_', 'alive': False, 'name': 'Robert', 'surname': 'Baratheon', 'traits': ['strong', 'powerful', 'loyal']}

{'_id': 'Characters/1026018920', '_key': '1026018920', '_rev': '_aplllqi--A', 'age': 36, 'alive': True, 'name': 'Jaime', 'surname': 'Lannister', 'traits': ['strong', 'experienced', 'polite']}

{'_id': 'Characters/1026018921', '_key': '1026018921', '_rev': '_aplllqi--B', 'age': 40, 'alive': False, 'name': 'Catelyn', 'surname': 'Stark', 'traits': ['beautiful', 'powerful', 'loyal']}

{'_id': 'Characters/1026018922', '_key': '1026018922', '_rev': '_aplllqi--C', 'age': 36, 'alive': True, 'name': 'Cersei', 'surname': 'Lannister', 'traits': ['powerful', 'sneaky', 'experienced']}



The [MERGE()](https://www.arangodb.com/docs/stable/aql/functions-document.html#merge) functions merges objects together. Because we used an object { traits: ... } which has the same attribute name traits as the original character attribute, the latter got overwritten by the merge operation.

# Joining Characters and Traits

The DOCUMENT() function utilizes primary indices to look up documents quickly. It is limited to find documents via their identifiers however. For a use case like in our example it is sufficient to accomplish a simple join.

There is another, more flexible syntax for joins: nested [FOR](https://www.arangodb.com/docs/stable/aql/operations-for.html) loops over multiple collections, with a [FILTER](https://www.arangodb.com/docs/stable/aql/operations-filter.html) condition to match up attributes. In case of the traits key array, there needs to be a third loop to iterate over the keys:

In [18]:
all_characters_traits = """
FOR c IN Characters
  LIMIT 5
  RETURN MERGE(c, {
    traits: (
      FOR key IN c.traits
        FOR t IN Traits
          FILTER t._key == key
          RETURN t.en
    )
  })
"""

query_result = db.AQLQuery(all_characters_traits, rawResults=True)
for doc in  query_result:
    print(doc)
    print()    

{'_id': 'Characters/1026018918', '_key': '1026018918', '_rev': '_aplllqi---', 'age': 41, 'alive': True, 'name': 'Ned', 'surname': 'Stark', 'traits': ['strong', 'powerful', 'loyal', 'rational', 'brave']}

{'_id': 'Characters/1026018919', '_key': '1026018919', '_rev': '_aplllqi--_', 'alive': False, 'name': 'Robert', 'surname': 'Baratheon', 'traits': ['strong', 'powerful', 'loyal']}

{'_id': 'Characters/1026018920', '_key': '1026018920', '_rev': '_aplllqi--A', 'age': 36, 'alive': True, 'name': 'Jaime', 'surname': 'Lannister', 'traits': ['strong', 'experienced', 'polite']}

{'_id': 'Characters/1026018921', '_key': '1026018921', '_rev': '_aplllqi--B', 'age': 40, 'alive': False, 'name': 'Catelyn', 'surname': 'Stark', 'traits': ['beautiful', 'powerful', 'loyal']}

{'_id': 'Characters/1026018922', '_key': '1026018922', '_rev': '_aplllqi--C', 'age': 36, 'alive': True, 'name': 'Cersei', 'surname': 'Lannister', 'traits': ['powerful', 'sneaky', 'experienced']}



For each character, it loops over its traits attribute (e.g. ["D","H","C"]) and for each document reference in this array, it loops over the Traits collections. There is a condition to match the document key with the key reference. The inner FOR loop and the FILTER get transformed to a primary index lookup in this case instead of building up a Cartesian product only to filter away everything but a single match: Document keys within a collection are unique, thus there can only be one match.

Each written-out, English trait is returned and all the traits are then merged with the character document. The result is identical to the query using DOCUMENT(). However, this approach with a nested FOR loop and a FILTER is not limited to primary keys. You can do this with any other attribute as well. For an efficient lookup, make sure you add a hash index for this attribute. If its values are unique, then also set the index option to unique.

# Next Steps

To continue playing and working with ArangoDB beyond the temporary database, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLJoin&utm_medium=Github&utm_campaign=ArangoDB%20University)
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at thttps://www.arangodb.com/arangodb-training-center/

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html